In [9]:
import gradio as gr
from ultralytics import YOLO
from PIL import Image
import numpy as np

# Cargar modelo entrenado
model = YOLO("/home/gnz/GitHub/yolo11_container/runs/detect/train/weights/best.pt")


In [ ]:
# Función de predicción con cropeo
def predict_with_crops(image):
    results = model.predict(image, conf=0.25, verbose=False)

    # Imagen anotada con bounding boxes
    img_with_boxes = results[0].plot()

    # Obtener crops de cada detección
    crops = []
    for box in results[0].boxes:
        # Extraer coordenadas
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        crop = image[y1:y2, x1:x2]  # crop sobre el array original
        crops.append(crop)

    # Convertir crops a lista de PIL images
    crop_images = [Image.fromarray(c) for c in crops]

    return img_with_boxes, crop_images

# Interfaz Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🚢 Container OCR Detector con Crops")
    gr.Markdown("Subí una imagen de un contenedor. El modelo dibuja los bounding boxes y devuelve también los recortes de cada detección.")

    with gr.Row():
        inp = gr.Image(type="numpy", label="Imagen de entrada")
    
    with gr.Row():
        out_img = gr.Image(type="numpy", label="Resultado con bounding boxes")
    
    with gr.Row():
        out_crops = gr.Gallery(label="Crops detectados", columns=2, height="auto")
        

    btn = gr.Button("Detectar y recortar")
    btn.click(fn=predict_with_crops, inputs=inp, outputs=[out_img, out_crops])

demo.launch(server_name="0.0.0.0", server_port=7865)

AttributeError: 'Gallery' object has no attribute 'style'